# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [45]:
data['ProductName'].value_counts()
y = data.groupby(['CustomerID','ProductName']).agg({'Quantity':sum})
display(y)

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [113]:
import numpy as np
tabla = pd.pivot_table(y, columns=['CustomerID'], values='Quantity', index=['ProductName'],aggfunc=np.sum)

In [114]:
tabla.fillna(value=0, inplace=True)
tabla

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [92]:
distances = pd.DataFrame(1/(1 + squareform(pdist(tabla.T, 'euclidean'))), 
                         index=tabla.columns, columns=tabla.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [107]:
distances.iloc[0].sort_values(ascending=False)[1:6]

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

In [306]:
similarities = distances[33].sort_values(ascending=False)[1:6]
list(similarities)
simil=list(similarities.index)
simil

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [180]:
tr=y.loc[[264,3535,3317,2503,3305]]
tr

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [247]:
a = tr.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
a

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [251]:
myMerge=a.merge(tabla[33], on='ProductName')
a1=myMerge.loc[myMerge[33]==0.0]
a1.head(5)

,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0
Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [398]:
dictEu={}
CustomerSetID=list(set(data['CustomerID']))
for e in CustomerSetID:
    similar = distances[e].sort_values(ascending=False)[1:6]
    simi=list(similar.index)
    table=y.loc[simi]
    am=table.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
    r =am.merge(tabla[e], on='ProductName')
    r=r.loc[r[e]==0.0]
    r=r.head(5)
    dictEu[e]=list(r.index)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [399]:
pd.DataFrame(dictEu).T

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
...,...,...,...,...,...
77814,Cheese - Victor Et Berthold,Cake - Cake Sheet Macaroon,Sea Bass - Whole,Squid U5 - Thailand,Guinea Fowl
32753,Grouper - Fresh,Sherry - Dry,"Chestnuts - Whole,canned",Napkin White - Starched,Sprouts - Alfalfa
20476,Soup - Campbells Tomato Ravioli,"Pasta - Detalini, White, Fresh","Cheese - Boursin, Garlic / Herbs",Beans - Kidney White,Wine - Blue Nun Qualitatswein
67582,Cake - Mini Cheesecake,Ecolab - Solid Fusion,Wine - Redchard Merritt,Mussels - Frozen,Cheese - Bocconcini


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [397]:
distCity = pd.DataFrame(1/(1 + squareform(pdist(tabla.T, 'cityblock'))), 
                         index=tabla.columns, columns=tabla.columns)
distCity

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.008000,0.009901,0.009174,0.008403,0.008929,0.007752,0.008850,0.008621,0.008696,...,0.000564,0.000596,0.000556,0.000674,0.000542,0.000569,0.000556,0.000554,0.000476,0.000534
200,0.008000,1.000000,0.008130,0.007874,0.006993,0.007576,0.007752,0.007752,0.008065,0.008130,...,0.000563,0.000593,0.000557,0.000675,0.000538,0.000567,0.000558,0.000553,0.000473,0.000531
264,0.009901,0.008130,1.000000,0.008547,0.008850,0.008475,0.007874,0.008403,0.008621,0.008264,...,0.000566,0.000596,0.000556,0.000678,0.000543,0.000576,0.000559,0.000557,0.000479,0.000533
356,0.009174,0.007874,0.008547,1.000000,0.008000,0.008197,0.007752,0.009174,0.008065,0.008130,...,0.000564,0.000595,0.000556,0.000676,0.000540,0.000571,0.000553,0.000556,0.000478,0.000534
412,0.008403,0.006993,0.008850,0.008000,1.000000,0.008065,0.007752,0.008850,0.007937,0.008264,...,0.000561,0.000599,0.000553,0.000674,0.000540,0.000570,0.000552,0.000555,0.000477,0.000536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.000569,0.000567,0.000576,0.000571,0.000570,0.000573,0.000571,0.000573,0.000572,0.000571,...,0.000336,0.000360,0.000322,0.000381,0.000357,1.000000,0.000364,0.000351,0.000298,0.000301
98069,0.000556,0.000558,0.000559,0.000553,0.000552,0.000554,0.000553,0.000556,0.000556,0.000556,...,0.000325,0.000336,0.000322,0.000342,0.000351,0.000364,1.000000,0.000351,0.000308,0.000330
98159,0.000554,0.000553,0.000557,0.000556,0.000555,0.000556,0.000554,0.000557,0.000558,0.000556,...,0.000330,0.000367,0.000333,0.000348,0.000328,0.000351,0.000351,1.000000,0.000317,0.000315


In [400]:
dictCity={}
CustomerSetID=list(set(data['CustomerID']))
for e in CustomerSetID:
    similar = distCity[e].sort_values(ascending=False)[1:6]
    simi=list(similar.index)
    table=y.loc[simi]
    am=table.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
    r =am.merge(tabla[e], on='ProductName')
    r=r.loc[r[e]==0.0]
    r=r.head(5)
    dictCity[e]=list(r.index)

In [401]:
pd.DataFrame(dictCity).T

,0,1,2,3,4
83973,"Pork - Bacon, Double Smoked","Nut - Chestnuts, Whole",Olive - Spread Tapenade,Butter - Unsalted,Beef Ground Medium
59399,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Pork - Bacon, Double Smoked",Soup - Campbells Bean Medley,"Salsify, Organic"
92168,Wine - Blue Nun Qualitatswein,Muffin Batt - Blueberry Passion,Knife Plastic - White,Chinese Foods - Chicken,"Wine - Magnotta, Merlot Sr Vqa"
49159,Wine - Blue Nun Qualitatswein,Butter - Unsalted,Tia Maria,Veal - Sweetbread,Wine - Chardonnay South
18441,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz,"Garlic - Primerba, Paste",Butter - Unsalted
...,...,...,...,...,...
77814,Wanton Wrap,Bandage - Fexible 1x3,Squid - Tubes / Tenticles 10/20,Lettuce - Spring Mix,Ice Cream Bar - Hageen Daz To
32753,Butter - Unsalted,Veal - Sweetbread,Tilapia - Fillets,Table Cloth 81x81 White,Hersey Shakes
20476,"Pasta - Detalini, White, Fresh",Knife Plastic - White,"Pepper - Black, Whole","Cheese - Boursin, Garlic / Herbs","Tart Shells - Sweet, 4"
67582,Lettuce - Spring Mix,Pork - Kidney,Halibut - Steaks,"Pasta - Detalini, White, Fresh","Veal - Inside, Choice"
